In [1]:
import joblib
from sklearn.metrics.pairwise import cosine_similarity
import redis
import pickle
import joblib
#from .models import JournalArticle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from django.shortcuts import get_object_or_404
redis_client = redis.Redis(host='containers-us-west-160.railway.app', port=5875,password="YSueGOq98SZCXvuet2ey")
JOURNAL_VECTORIZER = joblib.load('vectorizer.joblib')
JOURNAL_TFIDF_MATRIX = joblib.load('journal_tfidf.joblib')

journal_threshold = 3

def get_journal_index(user_input):
    user_tfidf = JOURNAL_VECTORIZER.transform([user_input])
    cosine_similarities = cosine_similarity(user_tfidf, JOURNAL_TFIDF_MATRIX).flatten()
    indices = cosine_similarities.argsort()[::-1]
    top_recommendations = [i for i in indices if cosine_similarities[i] > 0][:min(journal_threshold, len(indices))]
    print(top_recommendations)
    return top_recommendations


c:\Users\vukya\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\vukya\anaconda3\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Users\vukya\anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [4]:
def get_article_recommendations(user_input):
    l = []
    journal_ids = get_journal_index(user_input)
    for id in journal_ids:
        tfidf_matrix = joblib.load(f'journal_tfidf_{id}.joblib')
        vectorizer = joblib.load(f'vectorizer_{id}.joblib')

        user_tfidf = vectorizer.transform([user_input])
        cosine_similarities = cosine_similarity(user_tfidf, tfidf_matrix).flatten()
        
        indices = cosine_similarities.argsort()[::-1]
        top_recommendation_articles = [(cosine_similarities[i], i, id) for i in indices if cosine_similarities[i] > 0][:min(10, len(indices))]
        l.append(top_recommendation_articles)
    l.sort(reverse=True)
    return l



In [5]:

user_input = "AI"

print(get_article_recommendations(user_input))

[3, 2, 599]
[[(0.3634677801391609, 167, 3), (0.3335374973920005, 155, 3), (0.30212544862593343, 254, 3), (0.29350370171196033, 136, 3), (0.23531677328609785, 86, 3), (0.2174775839046739, 183, 3), (0.2174775839046739, 12, 3), (0.21321954198932952, 161, 3), (0.209885462803504, 290, 3), (0.20567041552733953, 85, 3)], [], []]
